<a href="https://colab.research.google.com/github/tgeorgy/rapid-detector/blob/master/examples/rapid_detector_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print("Cloning the rapid-detector repository...")
!git clone https://github.com/tgeorgy/rapid-detector.git

Cloning the rapid-detector repository...
fatal: destination path 'rapid-detector' already exists and is not an empty directory.


In [2]:
print("Installing dependencies...")
!cd rapid-detector/ && pip install -e .

Obtaining file:///content/rapid-detector
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/facebookresearch/sam3.git to /tmp/pip-install-sn5iz0ho/sam3_11e4ad95ff234b01be2bc73bb5bce564
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/sam3.git /tmp/pip-install-sn5iz0ho/sam3_11e4ad95ff234b01be2bc73bb5bce564
  Resolved https://github.com/facebookresearch/sam3.git to commit 99d02f28c8d34610ab35559b72642b01f10fdb1e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
print('Uninstalling numpy...')
!pip uninstall -y numpy
print('Installing numpy==1.26.4...')
!pip install numpy==1.26.4

Uninstalling numpy...
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Installing numpy==1.26.4...
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.13.0.90 requires numpy>=2; 

In [1]:
print('Logging in to Hugging Face Hub (this will prompt for your token)...')
from huggingface_hub import login
login()

Logging in to Hugging Face Hub (this will prompt for your token)...


In [1]:
print("Starting the server...")
import sys
sys.path.append('rapid-detector')
import threading
import uvicorn

def run_server():
    try:
        from app.api import app
        # host 0.0.0.0 is required for Colab's proxy to reach the service
        uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning")
    except Exception as e:
        print(f"❌ Error: {e}")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

In [ ]:
print("Waiting for the server to start...")
import time
time.sleep(20)

1. Name Object class
2. Upload Images
3. Check detections
4. Annotate a few examples
5. Click "Use API" and copy the detector_id, use it bellow

In [12]:
print("Opening the app UI...")
from google.colab import output
output.serve_kernel_port_as_iframe(8000, path='/', height=600)

<IPython.core.display.Javascript object>

In [42]:
print("Coppying uploaded images...")
!cp -r ~/.cache/rapid_detector/images .

In [19]:
print("Running inference...")
import glob
from matplotlib import pyplot
import requests

detector_id = "<YOUR DETECTOR ID FROM USE API WINDOW>"
api_url = f"http://localhost:8000/detect/{detector_id}"

for fpath in glob.glob('./images/*'):
    with open(fpath, "rb") as f:
        response = requests.post(api_url, files={"image": f})

    results = response.json()
    print(f"Found {len(results['boxes'])} detections")
    pyplot.imshow(pyplot.imread(fpath))
    for x0, y0, x1, y1 in results['boxes']:
        pyplot.plot([x0, x1, x1, x0, x0], [y0, y0, y1, y1, y0])
    pyplot.axis('off')
    pyplot.show()